In [1]:
import pandas as pd

In [2]:
walkins = pd.read_csv('walkins.csv', dtype = str)
users = pd.read_csv('users.csv', dtype = str)
pois = pd.read_csv('pois.csv', dtype = str)
categories = pd.read_csv('categories.csv', dtype = str)
points = pd.read_csv('users.csv', dtype = str)

In [3]:
categories.rename(columns = {'id': 'category_id', 'title': 'category'}, inplace = True)
pois.rename(columns = {'id': 'poi_id'}, inplace = True)
pois_categories = pd.merge(pois[['poi_id', 'category_id', 'title']],
                           categories,
                           how = 'left',
                           on = 'category_id')

### User Categories

In [4]:
visits = pd.merge(walkins[['created', 'poi_id', 'user_id']],
                  pois_categories[['poi_id', 'category_id', 'category']],
                  how = 'left',
                  on = 'poi_id')
visits = visits[['created', 'user_id', 'category']]

In [5]:
users = visits.groupby(by=['user_id', 'category']).count().reset_index()
users.rename(columns = {'created': 'visits'}, inplace = True)

#### Woman

* Beauty Store > 2
* Women's Store > 2

In [6]:
users['Woman'] = ((users['category'] == 'Women\'s Store') & (users['visits'] > 2)) | ((users['category'] == 'Beauty Store') & (users['visits'] > 2))


#### Parent

* Playground > 1
* Kids Store > 2

In [7]:
users['Parent'] = ((users['category'] == 'Playground') & (users['visits'] > 3)) | ((users['category'] == 'Kids Store') & (users['visits'] > 2))

Θα ήθελα να προσθέσω και αυτούς που έχουν πάει και στα 2 μέρη πάνω από 1 φορά.

#### Pet Owner

* Pet Store > 2

In [8]:
users['Pet Owner'] = ((users['category'] == 'Pet Store') & (users['visits'] > 2))

#### Gamer

* Gaming Cafe > 1

In [9]:
users['Gamer'] = ((users['category'] == 'Gaming Cafe') & (users['visits'] > 1))

#### Student

* University > 1

In [10]:
users['Student'] = ((users['category'] == 'University') & (users['visits'] > 1))

#### Athletic

* Gym > 3

In [11]:
users['Athletic'] = ((users['category'] == 'Gym') & (users['visits'] > 3))

#### Foodie

* Categoies: Souvlaki, Taverna, Steakhouse, Italian, Restaurant, Ethnic, Burgers, Sandwich Place
* Variety of categories visited > 3

In [12]:
users_foodie = users[(users['category'] == 'Souvlaki') |
                     (users['category'] == 'Taverna') |
                     (users['category'] == 'Steakhouse') |
                     (users['category'] == 'Italian') |
                     (users['category'] == 'Restaurant') |
                     (users['category'] == 'Ethnic') |
                     (users['category'] == 'Burgers') |
                     (users['category'] == 'Sandwich Place')]
users_foodie = users_foodie.groupby(by=['user_id']).count().reset_index()

In [13]:
users_foodie['variety'] = users_foodie['category'] > 3

In [14]:
users = pd.merge(users,
                 users_foodie[['user_id', 'variety']],
                 on = 'user_id',
                 how = 'left')

In [15]:
users['Foodie'] = False
users['Foodie'] = (users['variety'] == True)

#### Shopper

* Categoies: Women's Store, Mobile Phone Shop, Shoe Store, Clothing Store, Beauty Store, Game Store, Electronics Store, Kids Store, Shopping Mall, Home Store
* Variety of categories visited > 4

In [16]:
users_shopper = users[(users['category'] == ' Women\'s Store') |
                      (users['category'] == 'Mobile Phone Shop') |
                      (users['category'] == 'Shoe Store') |
                      (users['category'] == 'Clothing Store') |
                      (users['category'] == 'Beauty Store') |
                      (users['category'] == 'Game Store') |
                      (users['category'] == 'Electronics Store') | 
                      (users['category'] == 'Kids Store') |
                      (users['category'] == 'Shopping Mall') |
                      (users['category'] == 'Home Store')]
users_shopper = users_shopper.groupby(by=['user_id']).count().reset_index()

In [17]:
users_shopper['variety_2'] = users_shopper['category'] > 4

In [18]:
users = pd.merge(users,
                 users_shopper[['user_id', 'variety_2']],
                 on = 'user_id',
                 how = 'left')

In [19]:
users['Shopper'] = False
users['Shopper'] = (users['variety_2'] == True)

#### Final DataFrame

In [20]:
no_category = users[(users['Woman'] != True) & (users['Parent'] != True) & (users['Pet Owner'] != True) &
                    (users['Gamer'] != True) & (users['Student'] != True) & (users['Athletic'] != True) & 
                    (users['Foodie'] != True) & (users['Shopper'] != True)]
users = users[(users['Woman'] == True) | (users['Parent'] == True) | (users['Pet Owner'] == True) |
              (users['Gamer'] == True) | (users['Student'] == True) | (users['Athletic'] == True) | 
              (users['Foodie'] == True) | (users['Shopper'] == True)]
user_categories = users[['user_id', 'Woman', 'Parent', 'Pet Owner', 'Gamer', 'Student', 'Athletic', 'Foodie', 'Shopper']]
user_categories.drop_duplicates(inplace = True)
user_categories.replace({False: 0, True: 1}, inplace=True)
user_categories = user_categories.groupby(by=['user_id']).sum()

<ipython-input-20-e54053979ae9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_categories.drop_duplicates(inplace = True)
C:\Python\lib\site-packages\pandas\core\frame.py:4521: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


We only want values 0 and 1, where 0 is false and 1 is true.

In [21]:
user_categories.Foodie.unique()

array([0, 4, 3, 5, 6, 1], dtype=int64)

In [22]:
user_categories.Shopper.unique()

array([2, 4, 6, 3, 5, 0, 1], dtype=int64)

In [23]:
user_categories['Foodie'] = user_categories['Foodie'].replace([4, 3, 5, 6], 1)
user_categories['Shopper'] = user_categories['Shopper'].replace([2, 4, 6, 3, 5], 1)

### Amount of users by category

In [24]:
user_categories.sum().reset_index()

,index,0
0,Woman,707
1,Parent,46
2,Pet Owner,360
3,Gamer,27
4,Student,693
5,Athletic,346
6,Foodie,41
7,Shopper,689


### No category users

In [25]:
no_category

,user_id,category,visits,Woman,Parent,Pet Owner,Gamer,Student,Athletic,variety,Foodie,variety_2,Shopper
46,10037,Supermarket,1,False,False,False,False,False,False,NaN,False,NaN,False
47,10079,Home Store,1,False,False,False,False,False,False,NaN,False,False,False
75,10163,Cafe,4,False,False,False,False,False,False,NaN,False,NaN,False
76,10163,Supermarket,7,False,False,False,False,False,False,NaN,False,NaN,False
77,10185,Supermarket,1,False,False,False,False,False,False,NaN,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19639,9585,Metro Station,1,False,False,False,False,False,False,False,False,False,False
19640,9585,Playground,1,False,False,False,False,False,False,False,False,False,False
19641,9585,Sandwich Place,1,False,False,False,False,False,False,False,False,False,False
19642,9585,Supermarket,4,False,False,False,False,False,False,False,False,False,False


In [26]:
no_category.groupby(by = 'category').count().sort_values(by = 'user_id', ascending = False).head(10)

,user_id,visits,Woman,Parent,Pet Owner,Gamer,Student,Athletic,variety,Foodie,variety_2,Shopper
category,,,,,,,,,,,,
Supermarket,171,171,171,171,171,171,171,171,43,171,68,171
Park,130,130,130,130,130,130,130,130,23,130,49,130
Square,115,115,115,115,115,115,115,115,35,115,66,115
Metro Station,112,112,112,112,112,112,112,112,34,112,64,112
Cultural Center,108,108,108,108,108,108,108,108,22,108,43,108
Beach,103,103,103,103,103,103,103,103,19,103,39,103
Marina,87,87,87,87,87,87,87,87,12,87,24,87
Cafe,71,71,71,71,71,71,71,71,38,71,51,71
Landmark,63,63,63,63,63,63,63,63,4,63,23,63


In [27]:
no_category.groupby(by = 'user_id').count().sort_values(by = 'category', ascending = False).value_counts('category')

category
1     108
7      46
2      34
4      25
5      18
6      15
3      12
10      9
13      9
8       5
9       5
22      5
15      5
18      4
17      2
21      2
12      2
14      2
11      2
16      1
20      1
23      1
dtype: int64

### Overlapping categories

#### Woman

In [28]:
user_categories[user_categories['Woman'] == 1].sum().reset_index()

,index,0
0,Woman,707
1,Parent,39
2,Pet Owner,341
3,Gamer,24
4,Student,684
5,Athletic,338
6,Foodie,40
7,Shopper,679


In [62]:
women = user_categories[user_categories['Woman'] == 1]

In [63]:
women = women.reset_index()
wmn = pd.merge(women['user_id'],
               points,
               right_on = 'id',
               left_on = 'user_id',
               how = 'left')
wmn.points = wmn.points.astype(int)
wmn['points'].sum() / wmn['points'].count()

2790.6562942008486

In [64]:
wmn2 = pd.merge(wmn,
                 walkins,
                 on="user_id",
                 how="left")
wmn2=wmn2.groupby(by=["user_id"]).count()
wmn2["points"].sum()/wmn2["points"].count()

181.95615275813296

#### Parent

In [29]:
user_categories[user_categories['Parent'] == 1].sum().reset_index()

,index,0
0,Woman,39
1,Parent,46
2,Pet Owner,23
3,Gamer,3
4,Student,29
5,Athletic,19
6,Foodie,2
7,Shopper,35


#### Pet Owner

In [30]:
user_categories[user_categories['Pet Owner'] == 1].sum().reset_index()

,index,0
0,Woman,341
1,Parent,23
2,Pet Owner,360
3,Gamer,16
4,Student,332
5,Athletic,171
6,Foodie,20
7,Shopper,332


In [59]:
pOwner = user_categories[user_categories['Gamer'] == 1]

In [65]:
pOwner = pOwner.reset_index()
pwr = pd.merge(pOwner['user_id'],
               points,
               right_on = 'id',
               left_on = 'user_id',
               how = 'left')
pwr.points = pwr.points.astype(int)
pwr['points'].sum() / pwr['points'].count()

2940.5416666666665

In [61]:
pwr2 = pd.merge(pwr,
                 walkins,
                 on="user_id",
                 how="left")
pwr2=pwr2.groupby(by=["user_id"]).count()
pwr2["points"].sum()/pwr2["points"].count()

12.958333333333334

#### Gamer

In [31]:
user_categories[user_categories['Gamer'] == 1].sum().reset_index()

,index,0
0,Woman,24
1,Parent,3
2,Pet Owner,16
3,Gamer,27
4,Student,18
5,Athletic,17
6,Foodie,1
7,Shopper,21


In [56]:
gamer = user_categories[user_categories['Gamer'] == 1]

In [57]:
gamer = gamer.reset_index()
gmr = pd.merge(gamer['user_id'],
               points,
               right_on = 'id',
               left_on = 'user_id',
               how = 'left')
gmr.points = gmr.points.astype(int)
gmr['points'].sum() / gmr['points'].count()

3155.814814814815

In [58]:
gmr2 = pd.merge(gmr,
                 walkins,
                 on="user_id",
                 how="left")
gmr2=gmr2.groupby(by=["user_id"]).count()
gmr2["points"].sum()/gmr2["points"].count()

409.51851851851853

Parents

In [33]:
parent = user_categories[user_categories['Parent'] == 1]
parent.sum().reset_index()

,index,0
0,Woman,39
1,Parent,46
2,Pet Owner,23
3,Gamer,3
4,Student,29
5,Athletic,19
6,Foodie,2
7,Shopper,35


In [51]:
#parent = parent.reset_index()
parnt = pd.merge(parent['user_id'],
               points,
               right_on = 'id',
               left_on = 'user_id',
               how = 'left')
parnt.points = parnt.points.astype(int)
parnt['points'].sum() / parnt['points'].count()

3518.086956521739

In [39]:
parnt2 = pd.merge(parnt,
                 walkins,
                 on="user_id",
                 how="left")
parnt2=parnt2.groupby(by=["user_id"]).count()
parnt2["points"].sum()/parnt2["points"].count()

246.54347826086956

#### Student

In [42]:
student = user_categories[user_categories['Student'] == 1]

In [49]:
student = student.reset_index()
stud = pd.merge(student['user_id'],
               points,
               right_on = 'id',
               left_on = 'user_id',
               how = 'left')
stud.points = stud.points.astype(int)
stud['points'].sum() / stud['points'].count()

2728.784992784993

In [54]:
stud2 = pd.merge(stud,
                walkins,
                on = 'user_id',
                how = 'left')
stud2 = stud2.groupby(by = ['user_id']).count()
stud2['points'].sum() / stud2['points'].count()

169.14430014430013

#### Athletic

In [40]:
athletic = user_categories[user_categories['Athletic'] == 1]
athletic.sum().reset_index()

,index,0
0,Woman,338
1,Parent,19
2,Pet Owner,171
3,Gamer,17
4,Student,331
5,Athletic,346
6,Foodie,12
7,Shopper,330


In [41]:
athletic = athletic.reset_index()
ath = pd.merge(athletic['user_id'],
               points,
               right_on = 'id',
               left_on = 'user_id',
               how = 'left')
ath.points = ath.points.astype(int)
ath['points'].sum() / ath['points'].count()

3077.8121387283236

In [44]:
ath2 = pd.merge(ath,
                walkins,
                on = 'user_id',
                how = 'left')
ath2 = ath2.groupby(by = ['user_id']).count()
ath2['points'].sum() / ath2['points'].count()

196.17052023121389

#### Foodie

In [45]:
foodie = user_categories[user_categories['Foodie'] == 1]
foodie.sum().reset_index()

,index,0
0,Woman,40
1,Parent,2
2,Pet Owner,20
3,Gamer,1
4,Student,40
5,Athletic,12
6,Foodie,41
7,Shopper,38


In [46]:
foodie = foodie.reset_index()
food = pd.merge(foodie['user_id'],
                points,
                right_on = 'id',
                left_on = 'user_id',
               how = 'left')
food.points = food.points.astype(int)
food['points'].sum() / food['points'].count()

3220.829268292683

In [47]:
food2 = pd.merge(food,
                 walkins,
                 on = 'user_id',
                 how = 'left')
food2 = food2.groupby(by = ['user_id']).count()
food2['points'].sum() / food2['points'].count()

181.7560975609756

#### Shopper

In [48]:
shopper = user_categories[user_categories['Shopper'] == 1]
shopper.sum().reset_index()

,index,0
0,Woman,679
1,Parent,35
2,Pet Owner,332
3,Gamer,21
4,Student,677
5,Athletic,330
6,Foodie,38
7,Shopper,689


In [49]:
shopper = shopper.reset_index()
shop = pd.merge(shopper['user_id'],
                points,
                right_on = 'id',
                left_on = 'user_id',
               how = 'left')
shop.points = shop.points.astype(int)
shop['points'].sum() / shop['points'].count()

2783.644412191582

In [50]:
shop2 = pd.merge(shop,
                 walkins,
                 on = 'user_id',
                 how = 'left')
shop2 = shop2.groupby(by = ['user_id']).count()
shop2['points'].sum() / shop2['points'].count()

177.39187227866472